In [1]:
pip install slack_sdk

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

In [5]:
import csv
from openai import OpenAI
import os
import re

# OPENAI_API_KEYを含むとPushできないため実行時は有効にしてください
# os.environ["OPENAI_API_KEY"] = "*****"

# openAIの機能をclientに代入
client = OpenAI()

# 対象者のMBTIタイプ
user_mbti = "INTJ"

In [8]:
# CSVファイルを読み込み
with open('output.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    people = list(reader)

In [9]:
def find_best_matches(people, user_mbti, top_n=3):
    
    persons = ""
    for person in people:
        persons = persons + \
            str(f"【名前】 {person['Slack表示名']}, \
                【自己紹介】 {person['自己紹介']}, \
                【得意な業界】 {person['業界']}, \
                【関心のある領域】 {person['関心のある領域']}, \
                【協働で大切にしたいこと】 {person['Tech0のPJTで8期の仲間と協働する際に大切にしたいと思うこと']}, \
                【プロジェクト推進で得意なこと/苦手なこと】 {person['PJTをする上で自分が得意なこと・苦手なこと']}, \
                【参加動機や達成したいこと】 {person['Tech0の参加動機と１年後に到達したい・達成したいこと']},") + "\n"
    
    prompt = f"MBTIタイプが{user_mbti}の人物と合う人物を以下から3名選び、それぞれのMBTIタイプを推測してください。\
            また、対象者とペアでプロレスをする際の必殺技の名前も提案してください。\
            対象者は、次の形式に従って、出力してください。項目には余計な記号や装飾（**など）をつけず、名前はそのまま利用し漢字変換は行わないでください。\
            また、指定した形式に正確に従って出力してください。\
            形式の指定はじまり\n\
                【名前】[名前をそのまま出力]\n\
                【選定理由】[選定理由をそのまま出力]\n \
                【推定MBTI】[推定MBTIをそのまま出力]\n \
                【必殺技】[必殺技の名前をそのまま出力]\n \
            形式の指定終わり\n \
            ここから対象者の情報" + persons

    response =  client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", "content": prompt },
        ],
    )
    
    # コンテンツを抽出
    content = response.choices[0].message.content.strip()  
    print(content)
    
    # 対象者と選定理由を抽出するための正規表現パターン
    pattern = r"【名前】\s*(.*?)\s*【選定理由】\s*(.*?)\s*【推定MBTI】\s*(.*?)\s*【必殺技】\s*(.*?)\s*(?=【名前】|$)"
    matches = re.findall(pattern, content, re.DOTALL)

    # 結果を配列に格納
    result = [{"名前": match[0], "選定理由": match[1], "推定MBTI": match[2], "必殺技": match[3]} for match in matches]

    return result

NameError: name 'content' is not defined

In [10]:
# 最適な3人を選択
best_matches = find_best_matches(people, user_mbti)

# 結果を表示
for match in best_matches:
     print(match)

【名前】 Tnaka Yasuhiro-8  
【選定理由】 Tnakaは新しい技術を学び、自分でモックを作りたいという意欲を持っているので、テクノロジーに関心が高く、プロジェクトの推進や進行管理を重視するINTJと相性が良い。  
【推定MBTI】 ENFP  
【必殺技】 "未来の橋渡し"  

【名前】 Sugiyama Shoichi-8  
【選定理由】 Sugiyamaは決断力と進行の得意分野を持っており、リーダーシップを発揮できる要素があるため、INTJの戦略家としての特性を生かしやすい。  
【推定MBTI】 ENTJ  
【必殺技】 "決断の一撃"  

【名前】 Takahashi Hiroaki-8  
【選定理由】 Takahashiは資料作りの得意さから、情報の整理や構造化を重視する点でINTJと共通し、プロジェクトを進めるための策を考える隙間がある。  
【推定MBTI】 INTJ  
【必殺技】 "論理の渦"
{'名前': 'Tnaka Yasuhiro-8', '選定理由': 'Tnakaは新しい技術を学び、自分でモックを作りたいという意欲を持っているので、テクノロジーに関心が高く、プロジェクトの推進や進行管理を重視するINTJと相性が良い。', '推定MBTI': 'ENFP', '必殺技': '"未来の橋渡し"'}
{'名前': 'Sugiyama Shoichi-8', '選定理由': 'Sugiyamaは決断力と進行の得意分野を持っており、リーダーシップを発揮できる要素があるため、INTJの戦略家としての特性を生かしやすい。', '推定MBTI': 'ENTJ', '必殺技': '"決断の一撃"'}
{'名前': 'Takahashi Hiroaki-8', '選定理由': 'Takahashiは資料作りの得意さから、情報の整理や構造化を重視する点でINTJと共通し、プロジェクトを進めるための策を考える隙間がある。', '推定MBTI': 'INTJ', '必殺技': '"論理の渦"'}


In [16]:
best_matches[0]['名前']

'Tnaka Yasuhiro-8'

In [18]:
import streamlit as st

# Slackのユーザー名を指定する
user_name = best_matches[0]['名前']  # 送信先のSlackハンドル名

# ワークスペースIDを指定
workspace_id = "TXXXXXXXX"  # ワークスペースIDを指定

# Slack DMのDeep Linkを生成（ワークスペースを指定）
slack_deep_link = f"https://slack.com/app_redirect?team={workspace_id}&channel=@{user_name}"

# ボタンをクリックするとSlackのDMウィンドウが指定したワークスペースで立ち上がる
if st.button('SlackでDMを送信'):
    st.markdown(f"[{user_name}にDMを送る]({slack_deep_link})", unsafe_allow_html=True)